<a href="https://colab.research.google.com/github/busiri/busil/blob/main/3%EB%8B%A8%EA%B3%84(%EB%B6%84%EB%A6%AC%20%EB%B0%8F%20%EA%B2%B0%EC%B8%A1%EC%B9%98%20%EC%B2%98%EB%A6%AC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 데이터 로드

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from scipy.stats.mstats import winsorize
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
from tqdm import tqdm

phase_files = {
    '도입': 'eda도입.csv',
    '성장': 'eda성장.csv',
    '성숙': 'eda성숙.csv',
    '쇠퇴': 'eda쇠퇴.csv'
}

for phase, file in phase_files.items():
    df = pd.read_csv(file)
    print(f"[{phase}] 원본 데이터 shape: {df.shape}")
    # 결측치 개수 확인
    print(f"[{phase}] 결측치 개수:")
    print(df.isna().sum()[df.isna().sum() > 0])
    # 결측치 제거(행)
    df = df.dropna(axis=0, how='any')
    print(f"[{phase}] 결측치 제거 후 shape: {df.shape}")
    # train/test 분리
    train_df, test_df = train_test_split(
        df, test_size=0.2, random_state=42, stratify=df['부실여부']
    )
    train_df.to_csv(f'{phase}_train.csv', index=False)
    test_df.to_csv(f'{phase}_test.csv', index=False)

### 파생변수 생성

In [ ]:
def add_derived_vars_no_lag(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # 0) 로그 → 원본 복원 (필요 시)
    if '로그매출액' in df.columns:
        df['매출액(천원)'] = np.exp(df['로그매출액'])
        df.drop(columns=['로그매출액'], inplace=True)
    if '로그자산' in df.columns:
        df['자산(천원)'] = np.exp(df['로그자산'])
        df.drop(columns=['로그자산'], inplace=True)

    # 안전 나눗셈 도우미
    def safe_div(num, denom):
        return np.where(denom == 0, np.nan, num / denom)

    # --- 1) 수익성 지표
    df['/ 매출원가/매출액 비율']    = safe_div(df['매출원가(천원)'], df['매출액(천원)'])
    df['/ 사내유보율']            = safe_div(df['이익잉여금(천원)'], df['자본(천원)'])

    # --- 2) 활동성·효율성 지표
    net_wc = df['유동자산(천원)'] - df['유동부채(천원)']
    df['/ 재고/순운전자본 비율']   = safe_div(df['재고자산(천원)'], net_wc)
    df['/ 비유동부채/순운전자본 비율'] = safe_div(df['부채(천원)'] - df['유동부채(천원)'], net_wc)
    df['/ 자기자본회전률']        = safe_div(df['매출액(천원)'], df['자본(천원)'])
    df['/ 재고자산회전률']        = safe_div(df['매출원가(천원)'], df['재고자산(천원)'])
    df['/ 재고자산회전기간']      = safe_div(365, safe_div(df['매출원가(천원)'], df['재고자산(천원)']))
    df['/ 운전자본회전률']        = safe_div(df['매출액(천원)'], net_wc)
    df['/ 1회전기간']            = safe_div(365, safe_div(df['매출액(천원)'], net_wc))

    # --- 3) 인력·노동 생산성 지표
    df['/ 종업원1인당 부가가치(백만원)'] = safe_div(df['부가가치(백만원)'], df['종업원'])
    df['/ 종업원1인당 매출액(백만원)'] = safe_div(df['매출액(천원)'], df['종업원'])
    df['/ 종업원당영업이익(백만원)']   = safe_div(df['영업이익(손실)(천원)'], df['종업원']) / 1e3
    df['/ 종업원1인당 순이익(백만원)'] = safe_div(df['당기순이익(손실)(천원)'], df['종업원']) / 1e3
    df['/ 종업원1인당 인건비(백만원)'] = safe_div(df['급여(천원)'], df['종업원']) / 1e3
    df['/ 노동소득분배율']          = safe_div(df['급여(천원)'], df['부가가치(백만원)'] * 1e3)

    # --- 4) 자본집약도 지표
    df['/ 자본집약도']            = safe_div(df['자산(천원)'], df['종업원'])

    # --- 5) 기존 파생변수: 안정성·레버리지·현금흐름·투하자본수익률
    # 5-1) 안정성 비율
    df['/ 현금/총자산 비율']      = safe_div(df['현금및현금성자산(천원)'], df['자산(천원)'])
    df['/ 이익잉여금/총자산 비율'] = safe_div(df['이익잉여금(천원)'], df['자산(천원)'])
    df['/ 급여/총자산 비율']      = safe_div(df['급여(천원)'], df['자산(천원)'])

    # 5-2) 비용·레버리지·효율
    df['/ 급여/매출액 비율']      = safe_div(df['급여(천원)'], df['매출액(천원)'])
    df['/ 업력 효율지수']         = safe_div(df['매출액(천원)'], df['업력'])
    df['/ 매출원가/부채 비율']    = safe_div(df['매출원가(천원)'], df['부채(천원)'])
    df['/ 급여/세전순익 비율']    = safe_div(df['급여(천원)'], df['법인세비용차감전손익(천원)'])
    df['/ 재무 레버리지']        = safe_div(
        df['영업이익(손실)(천원)'],
        df['영업이익(손실)(천원)'] - df['이자비용(천원)']
    )

    # 5-3) 금융비용 부담률
    df['/ 금융비용부담률']        = safe_div(df['이자비용(천원)'], df['매출액(천원)'])

    # 5-4) 현금흐름 변수
    df['/ 영업CF/자산 비율']      = safe_div(df['영업현금흐름(천원)'], df['자산(천원)'])
    df['/ 투자CF/자산 비율']      = safe_div(df['투자활동으로 인한 현금흐름(*)(천원)'], df['자산(천원)'])
    df['/ 영업CF/유동부채 비율']   = safe_div(df['영업현금흐름(천원)'], df['유동부채(천원)'])
    # 총차입금이 없으면 부채(천원) 사용
    total_borrow = df['총차입금'] if '총차입금' in df.columns else df['부채(천원)']
    df['/ 재무CF/총차입금 비율']   = safe_div(df['재무활동으로 인한 현금흐름(*)(천원)'], total_borrow)

    # 5-5) 통합현금흐름 긍정여부
    cf_sum = (
        df['영업현금흐름(천원)']
      + df['투자활동으로 인한 현금흐름(*)(천원)']
      + df['재무활동으로 인한 현금흐름(*)(천원)']
    )
    df['/ 현금흐름통합_긍정여부'] = np.where(cf_sum > 0, 1, 0)
    df['/ 현금흐름통합_긍정여부'] = df['/ 현금흐름통합_긍정여부'].astype('object')
    # 5-6) 투하자본수익률 (영업이익 ÷ (자본+부채) ×100)
    total_cap = df['자본(천원)'] + df['부채(천원)']
    df['/ 투하자본수익률']       = safe_div(df['영업이익(손실)(천원)'], total_cap) * 100

    # 7) EBIT
    df['/ EBIT'] = df['법인세비용차감전손익(천원)'] + df['이자비용(천원)']
    df['/ EBIT / 자산'] = safe_div(df['/ EBIT'], df['자산(천원)'])
    df['/ EBIT / 매출액'] = safe_div(df['/ EBIT'], df['매출액(천원)'])
    df['/ EBIT / 영업현금흐름(천원)'] = safe_div(df['/ EBIT'], df['영업현금흐름(천원)'])

    # 8)
    df['/ 잉여현금흐름(FCF)'] = (df['영업현금흐름(천원)'] - df['투자활동으로 인한 현금흐름(*)(천원)'])
		df['/ 이익잉여금/부채 비율'] = safe_div(df['이익잉여금(천원)'],df['부채(천원)'])
		df['/ 잉여현금흐름 / 자산 비율'] = safe_div(df['/ 잉여현금흐름(FCF)'], df['자산(천원)'])

    # 9) 업력
    df['/ 업력_범주12'] = df['업력'].map(lambda x: 1 if x > 12 else 0)
    df['/ 업력_범주12'] = df['/ 업력_범주12'].astype('object')

    # 10) 무한대 → NaN
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # 파생변수에 사용된 컬럼 제거
    drop_cols = ['매출액(천원)','자산(천원)','매출원가(천원)','자본(천원)','유동자산(천원)', '유동부채(천원)','재고자산(천원)',
                '부채(천원)','유동부채(천원)','부가가치(백만원)','종업원','영업이익(손실)(천원)','당기순이익(손실)(천원)','급여(천원)',
                '업력','법인세비용차감전손익(천원)','이자비용(천원)','영업현금흐름(천원)','투자활동으로 인한 현금흐름(*)(천원)',
                '재무활동으로 인한 현금흐름(*)(천원)','/ EBIT','이익잉여금(천원)','/ 잉여현금흐름(FCF)','업력']

    df.drop(columns=drop_cols,inplace=True)
    return df


def make_variable(df1,df2):

  # 거래소코드와 회사명을 활용하여 기업ID 컬럼 생성
  df1 = df1.sort_values(by=['거래소코드', '회사명', '회계년도']).reset_index(drop=True)
  df2 = df2.sort_values(by=['거래소코드', '회사명', '회계년도']).reset_index(drop=True)
  df1['기업ID'] = df1['거래소코드'].astype(str) + "_" + df1['회사명'].astype(str)
  df2['기업ID'] = df2['거래소코드'].astype(str) + "_" + df2['회사명'].astype(str)

  # 수치형 컬럼 중 결측 적고 분산 큰 것 상위 100개 선정
  object_cols = df1.select_dtypes(include='object').columns
  df1_obj = df[object_cols]
  df2_obj = df[object_cols]
  num_cols = df1.select_dtypes(include=['float64', 'int64']).copy()
  valid_cols = num_cols.columns[num_cols.isnull().mean() < 0.3]
  sorted_cols = num_cols[valid_cols].var().sort_values(ascending=False).index
  top_100_cols = sorted_cols[:100].tolist()

  # 파생변수 생성
  for col in top_100_cols:
      try:
          print(f"Processing: {col}")
          grouped = df1.groupby('기업ID')
          grouped2 = df2.groupby('기업ID')
          c0 = grouped[col].shift(0)
          c1 = grouped[col].shift(1)
          c2 = grouped[col].shift(2)
          c3 = grouped[col].shift(3)
          test_c0 = grouped2[col].shift(0)
          test_c1 = grouped2[col].shift(1)
          test_c2 = grouped2[col].shift(2)
          test_c3 = grouped2[col].shift(3)

          df1[f'{col}_inc_2yr'] = ((c1 < c0) & (c2 < c1)).astype('object')
          df2[f'{col}_inc_2yr'] = ((test_c1 < test_c0) & (test_c2 < test_c1)).astype('object')

          df1[f'{col}_dec_2yr'] = ((c1 > c0) & (c2 > c1)).astype('object')
          df2[f'{col}_dec_2yr'] = ((test_c1 > test_c0) & (test_c2 > test_c1)).astype('object')

          df1[f'{col}_inc_3yr'] = ((c2 < c1) & (c1 < c0)).astype('object')
          df2[f'{col}_inc_3yr'] = ((test_c2 < test_c1) & (test_c1 < test_c0)).astype('object')

          df1[f'{col}_dec_3yr'] = ((c2 > c1) & (c1 > c0)).astype('object')
          df2[f'{col}_dec_3yr'] = ((test_c2 > test_c1) & (test_c1 > test_c0)).astype('object')

          df1[f'{col}_inc_4yr'] = ((c3 < c2) & (c2 < c1) & (c1 < c0)).astype('object')
          df2[f'{col}_inc_4yr'] = ((test_c3 < test_c2) & (test_c2 < test_c1) & (test_c1 < test_c0)).astype('object')

          df1[f'{col}_dec_4yr'] = ((c3 > c2) & (c2 > c1) & (c1 > c0)).astype('object')
          df2[f'{col}_dec_4yr'] = ((test_c3 > test_c2) & (test_c2 > test_c1) & (test_c1 > test_c0)).astype('object')

          diff1 = grouped[col].diff()
          diff2 = grouped2[col].diff()
          df1[f'{col}_mean_diff_3yr'] = diff1.rolling(3, min_periods=1).mean()
          df2[f'{col}_mean_diff_3yr'] = diff2.rolling(3, min_periods=1).mean()

          df1[f'{col}_pos_2yr'] = ((c1 > 0) & (c0 > 0)).astype('object')
          df2[f'{col}_pos_2yr'] = ((test_c1 > 0) & (test_c0 > 0)).astype('object')

          df1[f'{col}_dec_count'] = grouped1[col].diff1().lt(0).groupby(df1['기업ID']).cumsum().astype('object')
          df2[f'{col}_dec_count'] = grouped2[col].diff2().lt(0).groupby(df2['기업ID']).cumsum().astype('object')
      except Exception as e:
          print(f"⚠️ Error processing {col}: {e}")

  df1 = pd.concat([df1,df1_obj],axis=1)
  df2 = pd.concat([df2,df2_obj],axis=1)
  return df1, df2

for phase in phase_files:
    train_df = pd.read_csv(f'{phase}_train.csv')
    test_df = pd.read_csv(f'{phase}_test.csv')
    train_df_derived = add_derived_vars_no_lag(train_df)
    test_df_derived = add_derived_vars_no_lag(test_df)
    train_df_derived,test_df_derived = make_variable(train_df_derived,test_df_derived)
    train_df_derived.to_csv(f'{phase}_train_파생.csv', index=False)
    test_df_derived.to_csv(f'{phase}_test_파생.csv', index=False)

### 0 비율이 높은 컬럼 및 주식과 관련 된 컬럼 제거
- 외감기업이 대부분이라 신뢰성 있는 주식 관련 컬럼 데이터를 확보할 수 없다고
판단

In [ ]:
# 각 국면별 0비율이 높은 컬럼 제거

import pandas as pd

# 제거할 컬럼 리스트
cols_to_remove = [
    '대표이사변경여부', '상장폐지일', '종업원1인당 인건비(백만원)', '종업원1인당 순이익(백만원)',
    '종업원당영업이익(백만원)', '종업원1인당 매출액(백만원)', '종업원1인당 부가가치(백만원)',
    '종업원1인당 인건비증가율', '종업원1인당 매출액증가율', '종업원수증가율',
    '종업원1인당 부가가치증가율', '사내유보율', '배당성향', '노동장비율', '기계장비율','자본집약도','평균배당률','1주당매출액(원)','유보율','자기자본배당률'
]

phase_files = [
    ('도입_train_파생_dropna.csv', '도입'),
    ('성장_train_파생_dropna.csv', '성장'),
    ('성숙_train_파생_dropna.csv', '성숙'),
    ('쇠퇴_train_파생_dropna.csv', '쇠퇴')
]

for file, label in phase_files:
    df = pd.read_csv(file)
    # 실제로 존재하는 컬럼만 제거
    remove_cols = [col for col in cols_to_remove if col in df.columns]
    if remove_cols:
        print(f"[{label}] 제거할 컬럼: {remove_cols}")
        df = df.drop(columns=remove_cols)
    else:
        print(f"[{label}] 제거할 컬럼 없음")
    save_name = f"{label}_train_파생_컬럼제거.csv"
    df.to_csv(save_name, index=False)
    print(f"[{label}] 저장 완료: {save_name}")

### 이상치 처리 : 상/하위 1% 윈저라이징

In [ ]:
from scipy.stats.mstats import winsorize
import pandas as pd

def apply_winsorize(series, limits=(0.01, 0.01)):
    return winsorize(series, limits=limits)

for phase in ['도입', '성장', '성숙', '쇠퇴']:
    train_df_derived = pd.read_csv(f'{phase}_train_파생_컬럼제거.csv')
    # 수치형 컬럼 중 범주형 변수(cat_cols)와 '부실여부'는 제외
    cat_cols = train_df_derived.select_dtypes(include='object').columns
    num_cols = train_df_derived.select_dtypes(include=['int64', 'float64']).columns.tolist()
    num_cols = [col for col in num_cols if col != '부실여부' and col not in cat_cols]
    train_df_winsor = train_df_derived.copy()
    for col in num_cols:
        train_df_winsor[col] = apply_winsorize(train_df_derived[col], limits=(0.01, 0.01))
    train_df_winsor.to_csv(f'{phase}_train_파생_winsor.csv', index=False)

### 결측치 처리 : 회사 그룹별로 데이터가 있으면 회사 중앙값
- 결측치 문제를 완벽히 해결하지 못해서 이 방법을 쓰지는 않았습니다
- 결측치는 우선 0과 NaN 비율이 높은 컬럼을 제거한 뒤 행 단위 제거 방식으로 진행

## 수치형

In [ ]:
numeric_cols1 = train_도입.select_dtypes(include=[np.number]).columns
numeric_cols2 = train_성장.select_dtypes(include=[np.number]).columns
numeric_cols3 = train_성숙.select_dtypes(include=[np.number]).columns
numeric_cols4 = train_쇠퇴.select_dtypes(include=[np.number]).columns
print(f"도입기 훈련 데이터 결측치 수 : {train_도입[numeric_cols1].isna().sum().sum()}, 도입기 평가 데이터 결측치 수 : {test_도입[numeric_cols1].isna().sum().sum()}")
print(f"성장기 훈련 데이터 결측치 수 : {train_성장[numeric_cols2].isna().sum().sum()}, 성장기 평가 데이터 결측치 수 : {test_성장[numeric_cols2].isna().sum().sum()}")
print(f"성숙기 훈련 데이터 결측치 수 : {train_성숙[numeric_cols3].isna().sum().sum()}, 성숙기 평가 데이터 결측치 수 : {test_성숙[numeric_cols3].isna().sum().sum()}")
print(f"쇠퇴기 훈련 데이터 결측치 수 : {train_쇠퇴[numeric_cols4].isna().sum().sum()}, 쇠퇴기 평가 데이터 결측치 수 : {test_쇠퇴[numeric_cols4].isna().sum().sum()}")

도입기 훈련 데이터 결측치 수 : 2477, 도입기 평가 데이터 결측치 수 : 1155
성장기 훈련 데이터 결측치 수 : 4816, 성장기 평가 데이터 결측치 수 : 1440
성숙기 훈련 데이터 결측치 수 : 5087, 성숙기 평가 데이터 결측치 수 : 1801
쇠퇴기 훈련 데이터 결측치 수 : 1317, 쇠퇴기 평가 데이터 결측치 수 : 653


### 그룹화 후 중앙값으로 대체하는 방식(사용하지 못함)

In [ ]:
# tqdm 설정
tqdm.pandas()

def fill_na(df):
    group_cols = ['회사명', '거래소코드']
    numeric_cols = df.select_dtypes(include='number').columns

    # 그룹별 중앙값 계산
    group_medians = df.groupby(group_cols)[numeric_cols].median()

    # 각 행에 대해 결측치를 중앙값으로 채움 (단, 중앙값이 NaN이면 그대로 둠)
    def fill_with_group_median(row):
        key = (row['회사명'], row['거래소코드'])
        if key in group_medians.index:
            for col in numeric_cols:
                if pd.isna(row[col]):
                    median_val = group_medians.loc[key, col]
                    if not pd.isna(median_val):
                        row[col] = median_val
        return row

    # 진행률 표시하면서 적용
    df = df.progress_apply(fill_with_group_median, axis=1)
    return df

# 학습/테스트 리스트
result = []
for train, test in zip(train_list, test_list):
    train = fill_na(train)
    test = fill_na(test)
    result.append(train)
    result.append(test)

100%|██████████| 4796/4796 [00:03<00:00, 1467.19it/s]


In [ ]:
#위에서 0비율이 높으면서 주식 관련 컬럼 삭제 했음 행 단위로 결측치 제거
train_도입.dropna(axis=0,how='any',inplace=True)
train_성장.dropna(axis=0,how='any',inplace=True)
train_성숙.dropna(axis=0,how='any',inplace=True)
train_쇠퇴.dropna(axis=0,how='any',inplace=True)
test_도입.dropna(axis=0,how='any',inplace=True)
test_성장.dropna(axis=0,how='any',inplace=True)
test_성숙.dropna(axis=0,how='any',inplace=True)
test_쇠퇴.dropna(axis=0,how='any',inplace=True)
print(f"도입기 훈련 데이터 결측치 수 : {train_도입[numeric_cols1].isna().sum().sum()}, 도입기 평가 데이터 결측치 수 : {test_도입[numeric_cols1].isna().sum().sum()}")
print(f"성장기 훈련 데이터 결측치 수 : {train_성장[numeric_cols2].isna().sum().sum()}, 성장기 평가 데이터 결측치 수 : {test_성장[numeric_cols2].isna().sum().sum()}")
print(f"성숙기 훈련 데이터 결측치 수 : {train_성숙[numeric_cols3].isna().sum().sum()}, 성숙기 평가 데이터 결측치 수 : {test_성숙[numeric_cols3].isna().sum().sum()}")
print(f"쇠퇴기 훈련 데이터 결측치 수 : {train_쇠퇴[numeric_cols4].isna().sum().sum()}, 쇠퇴기 평가 데이터 결측치 수 : {test_쇠퇴[numeric_cols4].isna().sum().sum()}")

도입기 훈련 데이터 결측치 수 : 0, 도입기 평가 데이터 결측치 수 : 0
성장기 훈련 데이터 결측치 수 : 0, 성장기 평가 데이터 결측치 수 : 0
성숙기 훈련 데이터 결측치 수 : 0, 성숙기 평가 데이터 결측치 수 : 0
쇠퇴기 훈련 데이터 결측치 수 : 0, 쇠퇴기 평가 데이터 결측치 수 : 0


In [ ]:
# 기업규모명 삭제(0이 대부분이여서 제거함)
train_도입.drop(['대표이사변경여부','상장폐지일'],axis=1,inplace=True)
train_성장.drop(['대표이사변경여부','상장폐지일'],axis=1,inplace=True)
train_성숙.drop(['대표이사변경여부','상장폐지일'],axis=1,inplace=True)
train_쇠퇴.drop(['대표이사변경여부','상장폐지일'],axis=1,inplace=True)
test_도입.drop(['대표이사변경여부','상장폐지일'],axis=1,inplace=True)
test_성장.drop(['대표이사변경여부','상장폐지일'],axis=1,inplace=True)
test_성숙.drop(['대표이사변경여부','상장폐지일'],axis=1,inplace=True)
test_쇠퇴.drop(['대표이사변경여부','상장폐지일'],axis=1,inplace=True)

In [ ]:
object_cols1 = train_도입.select_dtypes('object').columns
object_cols2 = train_성장.select_dtypes('object').columns
object_cols3 = train_성숙.select_dtypes('object').columns
object_cols4 = train_쇠퇴.select_dtypes('object').columns
print(f"도입기 훈련 데이터 결측치 수 : {train_도입[object_cols1].isna().sum().sum()}, 도입기 평가 데이터 결측치 수 : {test_도입[object_cols1].isna().sum().sum()}")
print(f"성장기 훈련 데이터 결측치 수 : {train_성장[object_cols2].isna().sum().sum()}, 성장기 평가 데이터 결측치 수 : {test_성장[object_cols2].isna().sum().sum()}")
print(f"성숙기 훈련 데이터 결측치 수 : {train_성숙[object_cols3].isna().sum().sum()}, 성숙기 평가 데이터 결측치 수 : {test_성숙[object_cols3].isna().sum().sum()}")
print(f"쇠퇴기 훈련 데이터 결측치 수 : {train_쇠퇴[object_cols4].isna().sum().sum()}, 쇠퇴기 평가 데이터 결측치 수 : {test_쇠퇴[object_cols4].isna().sum().sum()}")

도입기 훈련 데이터 결측치 수 : 0, 도입기 평가 데이터 결측치 수 : 0
성장기 훈련 데이터 결측치 수 : 0, 성장기 평가 데이터 결측치 수 : 0
성숙기 훈련 데이터 결측치 수 : 0, 성숙기 평가 데이터 결측치 수 : 0
쇠퇴기 훈련 데이터 결측치 수 : 0, 쇠퇴기 평가 데이터 결측치 수 : 0


In [ ]:
# 파일 보내기
train_도입_파생.to_csv('도입기_train.csv',index=False)
train_성장_파생.to_csv('성장기_train.csv',index=False)
train_성숙_파생.to_csv('성숙기_train.csv',index=False)
train_쇠퇴_파생.to_csv('쇠퇴기_train.csv',index=False)
test_도입_파생.to_csv('도입기_test.csv',index=False)
test_성장_파생.to_csv('성장기_test.csv',index=False)
test_성숙_파생.to_csv('성숙기_test.csv',index=False)
test_쇠퇴_파생.to_csv('쇠퇴기_test.csv',index=False)